### Package imports

In [1]:
import math
import warnings
import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

### Warning suppress

In [2]:
warnings.filterwarnings('ignore')

### File reading

In [3]:
df = pd.read_csv('./dataset/realtor-data.csv')
df.head()

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [4]:
features = ['price','bed','bath','acre_lot','city','house_size']
df1 = df[features]
df1.head()

,price,bed,bath,acre_lot,city,house_size
0,105000.0,3.0,2.0,0.12,Adjuntas,920.0
1,80000.0,4.0,2.0,0.08,Adjuntas,1527.0
2,67000.0,2.0,1.0,0.15,Juana Diaz,748.0
3,145000.0,4.0,2.0,0.10,Ponce,1800.0
4,65000.0,6.0,2.0,0.05,Mayaguez,NaN


In [5]:
df1.shape

(733159, 6)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733159 entries, 0 to 733158
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   price       733088 non-null  float64
 1   bed         617177 non-null  float64
 2   bath        621407 non-null  float64
 3   acre_lot    572378 non-null  float64
 4   city        733087 non-null  object 
 5   house_size  520772 non-null  float64
dtypes: float64(5), object(1)
memory usage: 33.6+ MB


### Handling missing values

In [7]:
df1.isnull().sum()

price             71
bed           115982
bath          111752
acre_lot      160781
city              72
house_size    212387
dtype: int64

In [8]:
total_missing = df1.isnull().sum().sum()
total_cells = np.product(df1.shape)
missing_ratio = (total_missing/total_cells)*100
missing_ratio

13.663361790098282

In [9]:
df1[df1.isnull().any(axis=1)].drop('city',axis=1)

,price,bed,bath,acre_lot,house_size
4,65000.0,6.0,2.0,0.05,NaN
23,115000.0,3.0,2.0,NaN,1148.0
42,400000.0,NaN,NaN,0.99,43082.0
43,380000.0,NaN,NaN,18.45,NaN
51,160000.0,2.0,1.0,NaN,813.0
...,...,...,...,...,...
733153,459000.0,3.0,3.0,0.20,NaN
733154,748000.0,4.0,3.0,0.14,NaN
733155,649000.0,4.0,3.0,1.04,NaN
733156,300000.0,2.0,2.0,0.02,NaN


In [10]:
simple_imp = SimpleImputer(strategy='most_frequent')
df1[['city']] = simple_imp.fit_transform(df1[['city']])

In [11]:
iter_imp = IterativeImputer(max_iter=10, random_state=0)
iter_imp.fit(df1[~df1.isnull().any(axis=1)].drop('city',axis=1))

IterativeImputer(random_state=0)

In [12]:
imputed_data = iter_imp.transform(df1.drop('city',axis=1))
imputed_data

array([[1.05000000e+05, 3.00000000e+00, 2.00000000e+00, 1.20000000e-01,
        9.20000000e+02],
       [8.00000000e+04, 4.00000000e+00, 2.00000000e+00, 8.00000000e-02,
        1.52700000e+03],
       [6.70000000e+04, 2.00000000e+00, 1.00000000e+00, 1.50000000e-01,
        7.48000000e+02],
       ...,
       [3.00000000e+05, 2.00000000e+00, 2.00000000e+00, 2.00000000e-02,
        1.51356081e+03],
       [4.99900000e+05, 3.00000000e+00, 3.00000000e+00, 2.80000000e-01,
        2.01600000e+03],
       [3.89000000e+05, 4.00000000e+00, 1.00000000e+00, 1.30000000e-01,
        2.04093403e+03]])

In [13]:
df1[['price','bed','bath','acre_lot','house_size']] = imputed_data

In [14]:
df1.isnull().sum()

price         0
bed           0
bath          0
acre_lot      0
city          0
house_size    0
dtype: int64

### Handling data types

In [15]:
df1.loc[43]

price            380000.0
bed              3.796817
bath             2.553098
acre_lot            18.45
city              Hatillo
house_size    2211.015226
Name: 43, dtype: object

In [16]:
df1['bed'] = df1.bed.apply(lambda num: float(math.ceil(num)) if (math.ceil(num)-num)<=0.3 else float(math.floor(num)))
df1['bath'] = df1.bath.apply(lambda num: float(math.ceil(num)) if (math.ceil(num)-num)<=0.3 else float(math.floor(num)))
df1['house_size'] = df1.house_size.apply(lambda num: float(math.ceil(num)) if (math.ceil(num)-num)<=0.3 else float(math.floor(num)))

In [17]:
df1.loc[43]

price         380000.0
bed                4.0
bath               2.0
acre_lot         18.45
city           Hatillo
house_size      2211.0
Name: 43, dtype: object

### Feature engineering

In [18]:
df1['price_per_sqft'] = df1.price/df1.house_size
df1.head()

,price,bed,bath,acre_lot,city,house_size,price_per_sqft
0,105000.0,3.0,2.0,0.12,Adjuntas,920.0,114.130435
1,80000.0,4.0,2.0,0.08,Adjuntas,1527.0,52.390308
2,67000.0,2.0,1.0,0.15,Juana Diaz,748.0,89.572193
3,145000.0,4.0,2.0,0.10,Ponce,1800.0,80.555556
4,65000.0,6.0,2.0,0.05,Mayaguez,2671.0,24.335455


In [19]:
df1['sqft_lot'] = df1.acre_lot * 43560
df1.head()

,price,bed,bath,acre_lot,city,house_size,price_per_sqft,sqft_lot
0,105000.0,3.0,2.0,0.12,Adjuntas,920.0,114.130435,5227.2
1,80000.0,4.0,2.0,0.08,Adjuntas,1527.0,52.390308,3484.8
2,67000.0,2.0,1.0,0.15,Juana Diaz,748.0,89.572193,6534.0
3,145000.0,4.0,2.0,0.10,Ponce,1800.0,80.555556,4356.0
4,65000.0,6.0,2.0,0.05,Mayaguez,2671.0,24.335455,2178.0


In [20]:
df1['house-to-land'] = df1.house_size/df1.sqft_lot
df1.head()

,price,bed,bath,acre_lot,city,house_size,price_per_sqft,sqft_lot,house-to-land
0,105000.0,3.0,2.0,0.12,Adjuntas,920.0,114.130435,5227.2,0.176002
1,80000.0,4.0,2.0,0.08,Adjuntas,1527.0,52.390308,3484.8,0.438189
2,67000.0,2.0,1.0,0.15,Juana Diaz,748.0,89.572193,6534.0,0.114478
3,145000.0,4.0,2.0,0.10,Ponce,1800.0,80.555556,4356.0,0.413223
4,65000.0,6.0,2.0,0.05,Mayaguez,2671.0,24.335455,2178.0,1.226354


In [21]:
city_stats = df1.groupby('city')['city'].agg('count').sort_values(ascending=False)
city_stats

city
Boston                 31574
New York City          15447
New York               12981
Philadelphia           10416
Brooklyn                6914
                       ...  
Montrose                   1
Monhegan Island Plt        1
Edinburg                   1
Mongaup Valley             1
Cuddebackville             1
Name: city, Length: 2438, dtype: int64

In [22]:
city_stats_less_than_10 = city_stats[city_stats<=10]

In [23]:
city_stats_less_than_10

city
Cross Lake Township    10
Belleplain             10
Stacyville             10
South Setauket         10
Dresher                10
                       ..
Montrose                1
Monhegan Island Plt     1
Edinburg                1
Mongaup Valley          1
Cuddebackville          1
Name: city, Length: 390, dtype: int64

In [24]:
df1.city = df1.city.apply(lambda name: 'Other' if name in city_stats_less_than_10 else name)

### Outlier detection and removal

In [25]:
df1.describe()

,price,bed,bath,acre_lot,house_size,price_per_sqft,sqft_lot,house-to-land
count,7.331590e+05,733159.000000,733159.000000,733159.000000,7.331590e+05,733159.000000,7.331590e+05,7.331590e+05
mean,7.257003e+05,3.496663,2.476130,17.544873,2.204890e+03,328.240372,7.642547e+05,inf
std,1.768683e+06,1.885422,1.863615,903.023786,2.690444e+03,564.127407,3.933572e+07,NaN
min,0.000000e+00,1.000000,0.000000,-43.809700,1.000000e+02,0.000000,-1.908351e+06,-1.090466e+00
25%,2.375000e+05,3.000000,2.000000,0.180000,1.341000e+03,143.244626,7.840800e+03,4.941383e-03
50%,4.190000e+05,3.000000,2.000000,0.730000,2.000000e+03,229.789272,3.179880e+04,6.716651e-02
75%,7.250000e+05,4.000000,3.000000,10.463638,2.463000e+03,359.687653,4.557961e+05,2.431990e-01
max,8.750000e+08,123.000000,198.000000,100000.000000,1.450112e+06,358606.557377,4.356000e+09,inf


* due to lacks of info about floors & other rooms => cannot inspect related problems (ex: less house size but more bedrooms)
* price,bath = 0.0 => price cannot be lower than 50k, at least 1 bathroom
* acre_lot < 0
* house_size > sqft_lot
* bath > bed+2
* house-to-land ratio should be 3:5 => at least 40% unused land and 20% house area
* too high acre_lot => max sqft_lot is 100k
* minimum bedroom size is 300 sqft
* huge gap between min and max price

In [26]:
df2 = df1[(df1.price >= 50000) & (df1.bath > 0)]
df2.shape

(710854, 9)

In [27]:
df3 = df2[df2.acre_lot > 0]
df3.shape

(710200, 9)

In [28]:
df4 = df3[df3.house_size < df3.sqft_lot]
df4.shape

(674791, 9)

In [29]:
df5 = df4[df4.bed+2 >= df4.bath]
df5.shape

(671362, 9)

In [30]:
df6 = df5[(1-df5['house-to-land']>=0.4) & (df5['house-to-land']>=0.2)]
df6.shape

(136286, 9)

In [31]:
df7 = df6[df6.sqft_lot <= 100000]
df7.shape

(136284, 9)

In [32]:
df8 = df7[df7.house_size/df7.bed >= 300]
df8.shape

(132994, 9)

In [33]:
def price_outlier_removal(df):
    df_out = pd.DataFrame()
    for city, city_df in df.groupby('city'):
        m = np.mean(city_df.price)
        std = np.std(city_df.price)
        reduced_df = city_df[(city_df.price>m-std) & (city_df.price<=m+std)]
        df_out = pd.concat([df_out, reduced_df],ignore_index=True)
    return df_out

In [34]:
df9 = price_outlier_removal(df8)
df9.shape

(98553, 9)

In [35]:
df9.describe()

,price,bed,bath,acre_lot,house_size,price_per_sqft,sqft_lot,house-to-land
count,9.855300e+04,98553.000000,98553.000000,98553.000000,98553.000000,98553.000000,98553.000000,98553.000000
mean,5.668813e+05,3.805546,2.499396,0.156103,2170.786582,262.758045,6799.829460,0.344483
std,5.283109e+05,1.432986,1.095713,0.080129,952.944555,183.623779,3490.407262,0.109226
min,5.000000e+04,1.000000,1.000000,0.030000,400.000000,11.982571,1306.800000,0.200003
25%,2.899000e+05,3.000000,2.000000,0.110000,1568.000000,155.577173,4791.600000,0.251377
50%,4.250000e+05,4.000000,2.000000,0.140000,2000.000000,223.274696,6098.400000,0.319865
75%,6.499000e+05,4.000000,3.000000,0.190000,2503.000000,311.698718,8276.400000,0.424284
max,9.995000e+06,21.000000,20.000000,2.010000,19998.000000,2935.294118,87555.600000,0.599939


In [36]:
df10 = df9.drop(['price_per_sqft','acre_lot','house-to-land'],axis=1)
df10.head()

,price,bed,bath,city,house_size,sqft_lot
0,495000.0,3.0,4.0,Aberdeen,2146.0,6969.6
1,380000.0,3.0,2.0,Aberdeen,1872.0,6098.4
2,499900.0,4.0,3.0,Aberdeen,2165.0,4356.0
3,555000.0,3.0,3.0,Aberdeen,1692.0,7405.2
4,564900.0,4.0,3.0,Aberdeen,1860.0,7840.8


### Label encoding

In [37]:
city_dummies = pd.get_dummies(df9.city,dtype=float).drop('Other',axis=1)

In [38]:
city_dummies.shape

(98553, 1058)

In [39]:
df11 = pd.concat([df10,city_dummies],axis=1)
df11.head()

,price,bed,bath,city,house_size,sqft_lot,Aberdeen,Abington,Absecon,Acton,...,Wyckoff,Wyndmoor,Wynnewood,Yabucoa,Yardley,Yarmouth,Yauco,Yeadon,Yonkers,York
0,495000.0,3.0,4.0,Aberdeen,2146.0,6969.6,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,380000.0,3.0,2.0,Aberdeen,1872.0,6098.4,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,499900.0,4.0,3.0,Aberdeen,2165.0,4356.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,555000.0,3.0,3.0,Aberdeen,1692.0,7405.2,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,564900.0,4.0,3.0,Aberdeen,1860.0,7840.8,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
df12 = df11.drop('city',axis=1)
df12.head()

,price,bed,bath,house_size,sqft_lot,Aberdeen,Abington,Absecon,Acton,Adams,...,Wyckoff,Wyndmoor,Wynnewood,Yabucoa,Yardley,Yarmouth,Yauco,Yeadon,Yonkers,York
0,495000.0,3.0,4.0,2146.0,6969.6,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,380000.0,3.0,2.0,1872.0,6098.4,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,499900.0,4.0,3.0,2165.0,4356.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,555000.0,3.0,3.0,1692.0,7405.2,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,564900.0,4.0,3.0,1860.0,7840.8,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df12.shape

(98553, 1063)

### Model building

In [42]:
X = df12.drop('price',axis=1)
X.head()

,bed,bath,house_size,sqft_lot,Aberdeen,Abington,Absecon,Acton,Adams,Agawam,...,Wyckoff,Wyndmoor,Wynnewood,Yabucoa,Yardley,Yarmouth,Yauco,Yeadon,Yonkers,York
0,3.0,4.0,2146.0,6969.6,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,1872.0,6098.4,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,3.0,2165.0,4356.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,3.0,1692.0,7405.2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,1860.0,7840.8,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
y = df12.price
y.head()

0    495000.0
1    380000.0
2    499900.0
3    555000.0
4    564900.0
Name: price, dtype: float64

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [45]:
def finding_the_best_model(X,y):
    algos = {
        'linear': {
            'class': LinearRegression(),
            'params': {
                'n_jobs': [1,-1]
            }
        },
        'lasso': {
            'class': Lasso(random_state=0),
            'params': {
                'alpha': [1,2],
                'selection': ['cyclic','random']
            }
        },
        'tree': {
            'class': DecisionTreeRegressor(random_state=0),
            'params': {
                'criterion': ['squared_error','friedman_mse'],
                'splitter': ['random','best']
            }
        }
    }

    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for name, model in algos.items():
        gs = GridSearchCV(model['class'],model['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [46]:
finding_the_best_model(X[:5000],y[:5000])

,model,best_score,best_params
0,linear,0.901965,{'n_jobs': 1}
1,lasso,0.901962,"{'alpha': 1, 'selection': 'random'}"
2,tree,0.993431,"{'criterion': 'squared_error', 'splitter': 'be..."


In [47]:
tree = DecisionTreeRegressor(criterion='squared_error',splitter='random',random_state=0)
tree.fit(X_train, y_train)
tree.score(X_test, y_test)

0.9822284788890676

In [48]:
def price_prediction(city,bed,bath,house_size,land_size):
    try:
        city_index = np.where(X.columns==city)[0][0]
    except:
        city_index = -1

    x = np.zeros(len(X.columns))
    x[0] = bed
    x[1] = bath
    x[2] = house_size
    x[3] = land_size
    if city_index >=0:
        x[city_index] = 1
    return tree.predict([x])[0]

In [50]:
price_prediction('New York City',2,2,2000,4000)

2850000.0

In [51]:
df11[df11.city == 'New York City']

,price,bed,bath,city,house_size,sqft_lot,Aberdeen,Abington,Absecon,Acton,...,Wyckoff,Wyndmoor,Wynnewood,Yabucoa,Yardley,Yarmouth,Yauco,Yeadon,Yonkers,York
53697,735000.0,1.0,1.0,New York City,1257.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53698,4100000.0,3.0,3.0,New York City,3007.0,6969.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53699,550000.0,1.0,1.0,New York City,510.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53700,550000.0,1.0,1.0,New York City,510.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53701,550000.0,1.0,1.0,New York City,531.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54300,3195000.0,2.0,2.0,New York City,2953.0,12196.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54301,3995000.0,2.0,3.0,New York City,2025.0,7840.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54302,1591380.0,1.0,2.0,New York City,883.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54303,2250000.0,2.0,3.0,New York City,1252.0,2178.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Model export

In [52]:
with open('../server/model/home_price_model.pickle','wb') as f:
    pickle.dump(tree,f)

In [53]:
columns = {
    'data_cols': [col.lower() for col in X.columns]
}
with open('../server/model/columns.json','w') as f:
    f.write(json.dumps(columns))

In [54]:
city_columns = {
    'data_cols': [col.lower() for col in city_dummies.columns]
}
with open('../server/model/city_columns.json','w') as f:
    f.write(json.dumps(city_columns))